In [1]:
import wandb
wandb.login()
import os
os.environ['WANDB_PROJECT'] = 'shortest_v3_model'
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import csrgraph as cg
import numpy as np
import random
import string
from itertools import combinations
import pickle
import gc
import os
import json
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm


#os.environ['WANDB_MODE'] = 'disabled'

# set current directory
os.chdir("/cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS")

wandb: Currently logged in as: carobgt (carobgt-ucl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
/cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
import shutil
import random
import networkx as nx
import numpy as np
import string
from tqdm import tqdm

def get_graph_with_random_names(size):
    """Creates a grid graph with unique random 2-letter names."""
    num_nodes = size * size
    names = set()
    while len(names) < num_nodes:
        names.add(''.join(random.choices(string.ascii_lowercase, k=2)))
    
    nodes = list(names)
    random.shuffle(nodes) # Important for breaking name predictability
    
    G = nx.DiGraph()
    for r in range(size):
        for c in range(size):
            idx = r * size + c
            u = nodes[idx]
            if c < size - 1: G.add_edge(u, nodes[idx + 1], direction='EAST')
            if c > 0: G.add_edge(u, nodes[idx - 1], direction='WEST')
            if r < size - 1: G.add_edge(u, nodes[idx + size], direction='SOUTH')
            if r > 0: G.add_edge(u, nodes[idx - size], direction='NORTH')
    return G

def generate_random_walk(G, start_node, length):
    """Generates a simple random walk allowing backtracking."""
    path = [start_node]
    current_node = start_node
    while len(path) < length:
        neighbors = list(G.successors(current_node))
        if not neighbors: break
        next_node = random.choice(neighbors)
        path.append(next_node)
        current_node = next_node
    return path

def format_path_to_string(path, G):
    """Converts a list of nodes into a 'NODE DIR NODE...' string."""
    if not path or len(path) < 2: return path[0] if path else ""
    path_str = ""
    for i in range(len(path) - 1):
        u, v = path[i], path[i+1]
        try:
            direction = G.edges[u, v]['direction']
            path_str += f"{u} {direction} "
        except KeyError:
            # Safety check, although this should not be triggered with valid paths
            print(f"Warning: Edge ({u}, {v}) not found in graph. Skipping segment.")
            continue
    path_str += path[-1]
    return path_str


def generate_geometer_example(grid_size, min_walk_len, max_walk_len, max_walk_attempts=100000):
    """
    Generates a single example for the Geometer model using the
    "Persistent Filtering" method, correctly handling multiple shortest paths.
    """
    # Pick task first
    G = get_graph_with_random_names(grid_size)
    nodes = list(G.nodes())
    start_node, goal_node = random.sample(nodes, 2)
    
    try:
        # Find ALL shortest paths and their required node sets.
        all_solution_paths = list(nx.all_shortest_paths(G, source=start_node, target=goal_node))
        if not all_solution_paths:
            return None
        
        # This is now a list of sets of nodes, one for each valid shortest path.
        list_of_required_nodes = [set(p) for p in all_solution_paths]
        
    except (nx.NetworkXNoPath, StopIteration):
        return None # Should not happen on a connected grid

    # Brute-force a valid context.
    for _ in range(max_walk_attempts):
        walk_len = random.randint(min_walk_len, max_walk_len)
        walk_start_node = random.choice(nodes)
        context_walk = generate_random_walk(G, walk_start_node, walk_len)
        
        nodes_in_walk = set(context_walk)
        
        # Find which shortest paths are actually supported by this context
        supported_paths = []
        for i, req_nodes in enumerate(list_of_required_nodes):
            if req_nodes.issubset(nodes_in_walk):
                supported_paths.append(all_solution_paths[i])
        
        # Check if at least one path is supported
        if supported_paths:
            # Pick a target from only the supported paths
            ground_truth_path = random.choice(supported_paths)
            
            context_str = format_path_to_string(context_walk, G)
            target_str = format_path_to_string(ground_truth_path, G)
            
            task_instruction = f"[SHORTEST] [START_NODE] {start_node} [GOAL] {goal_node}"
            prompt = f"[SOS] {context_str} [SEP] {task_instruction} [PLAN]"
            target = f"{target_str} [EOS]"
            
            return prompt, target
    
    return None


def generate_and_save_dataset(n_examples, grid_size, min_walk_len, max_walk_len, output_file_path, description=""):
    """Orchestrates generating and saving the full dataset."""
    print(f"--- Generating {n_examples} examples for {description} ---")
    
    with open(output_file_path, "w", encoding="utf-8") as f:
        pbar = tqdm(total=n_examples, desc=f"Generating {description}")
        generated_count = 0
        attempts = 0
        max_total_attempts = n_examples * 1000  # Prevent infinite loops
        
        while generated_count < n_examples and attempts < max_total_attempts:
            attempts += 1
            example = generate_geometer_example(grid_size, min_walk_len, max_walk_len)
            
            if example:
                prompt, target = example
                f.write(f"{prompt} {target}\n")
                generated_count += 1
                pbar.update(1)
        
        pbar.close()
        
        if attempts >= max_total_attempts:
            print(f"Warning: Hit maximum attempts ({max_total_attempts}). Generated {generated_count}/{n_examples} examples.")
        else:
            success_rate = generated_count / attempts * 100
            print(f"Success rate: {success_rate:.2f}% ({generated_count}/{attempts} attempts)")

In [ ]:
if __name__ == "__main__":

    NUM_TRAIN_EXAMPLES = 1000000
    NUM_TEST_EXAMPLES = 10000
    GRID_SIZE = 4
    
    MIN_WALK_LENGTH = 10
    MAX_WALK_LENGTH = 50

    random.seed(42)
    np.random.seed(42)

    output_dir = "sv3_data"
    if os.path.exists(output_dir):
        print(f"Removing existing directory: {output_dir}")
        shutil.rmtree(output_dir)
    print(f"Creating new directory: {output_dir}")
    os.makedirs(output_dir)

    train_file_path = os.path.join(output_dir, "train.txt")
    generate_and_save_dataset(
        n_examples=NUM_TRAIN_EXAMPLES,
        grid_size=GRID_SIZE,
        min_walk_len=MIN_WALK_LENGTH,
        max_walk_len=MAX_WALK_LENGTH,
        output_file_path=train_file_path,
        description="Training Set"
    )


    test_file_path = os.path.join(output_dir, "test.txt")
    generate_and_save_dataset(
        n_examples=NUM_TEST_EXAMPLES,
        grid_size=GRID_SIZE,
        min_walk_len=MIN_WALK_LENGTH,
        max_walk_len=MAX_WALK_LENGTH,
        output_file_path=test_file_path,
        description="Testing Set"
    )

    print(f"\n--- SV3 data generation using 'Persistent Filtering' finished successfully! ---")



Removing existing directory: sv3_data
Creating new directory: sv3_data
--- Generating 1000000 examples for Training Set ---


Generating Training Set:   0%|          | 0/1000000 [00:00<?, ?it/s]

Generating Training Set: 100%|██████████| 1000000/1000000 [01:45<00:00, 9505.75it/s]


Success rate: 100.00% (1000000/1000000 attempts)
--- Generating 10000 examples for Testing Set ---


Generating Testing Set: 100%|██████████| 10000/10000 [00:01<00:00, 9579.47it/s]

Success rate: 100.00% (10000/10000 attempts)

--- SV3 data generation using 'Persistent Filtering' finished successfully! ---


In [27]:
# Analysis tools
from transformers import GPT2Tokenizer
import numpy as np
def analyse_dataset(file_path, tokenizer_path):
    """
    Analyze context/target ratio in existing dataset.
    """
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    
    total_context_tokens = 0
    total_target_tokens = 0
    ratios = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 1000:  # Analyze first 1000 examples
                break
                
            parts = line.strip().split()
            sep_index = parts.index('[PLAN]')
            prompt = ' '.join(parts[:sep_index])
            target = ' '.join(parts[sep_index + 1:])

            
            # Extract context (before [SEP])
            if '[SEP]' in prompt:
                context_part = prompt.split('[SEP]')[0]
            else:
                context_part = prompt
            
            context_tokens = len(tokenizer.encode(context_part))
            target_tokens = len(tokenizer.encode(target))
            
            if target_tokens > 0:
                ratio = context_tokens / target_tokens
                ratios.append(ratio)
                total_context_tokens += context_tokens
                total_target_tokens += target_tokens
    
    if ratios:
        avg_ratio = total_context_tokens / total_target_tokens
        median_ratio = np.median(ratios)
        
        print(f"Dataset Analysis ({len(ratios)} examples):")
        print(f"Average context/target ratio: {avg_ratio:.2f}")
        print(f"Median ratio: {median_ratio:.2f}")
        print(f"Ratio range: {min(ratios):.2f} - {max(ratios):.2f}")
        
        if avg_ratio > 3.0:
            print("Context heavily dominates - consider shorter contexts")
        elif avg_ratio < 0.5:
            print("Target dominates - consider longer contexts") 
        else:
            print("Reasonable balance")

analyse_dataset("sv3_data/train.txt", "gv6_tokenizer")

Dataset Analysis (1000 examples):
Average context/target ratio: 8.82
Median ratio: 9.00
Ratio range: 1.88 - 21.43
Context heavily dominates - consider shorter contexts


**CUSTOM TOKENIZER**

trained in tokenizer_training.ipynb


In [2]:
class GPT:

    def __init__(self, base_model=None, base_model_name='gpt2', vocab_size=100):
        self.base_model = base_model
        self.base_model_name = base_model_name
        self.vocab_size = vocab_size

        if self.base_model is not None:
            self.tokenizer = GPT2Tokenizer.from_pretrained(base_model)
            self.model = GPT2LMHeadModel.from_pretrained(base_model)
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def continue_input(self, input_sequence, max_new_tokens=5, num_return_sequences=1, no_repeat_ngram_size=0,
                       do_sample=False, temperature=0.7, num_beams=1):
        input_ids = self.tokenizer.encode(input_sequence, return_tensors='pt')

        # Generate text
        output = self.model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            num_return_sequences=num_return_sequences,
            num_beams=num_beams,
            no_repeat_ngram_size=no_repeat_ngram_size,
            do_sample=do_sample,
            temperature=temperature,
        )

        # Decode the output
        sequence = output[0].tolist()
        text = self.tokenizer.decode(sequence)
        return text
    
    

In [3]:
def load_pkl(pth):
    with open(pth, 'rb') as f:
        d = pickle.load(f)
    return d

def is_valid_path(sequence, graphs):
    # Split the sequence into parts
    parts = sequence.split()

    # Extract nodes and edges; nodes are at even indices, edges at odd indices
    nodes = parts[::2]
    edges = parts[1::2]

    # Convert edges to a lowercase version for comparison (assuming all edges in graphs are lowercase)
    edges = [edge.lower() for edge in edges]

    # Iterate over each graph to check if the path exists
    for graph in graphs:
        path_exists = True
        for i in range(len(nodes) - 1):
            # Check if the current graph has the edge between the current node and the next node
            if not graph.has_edge(nodes[i], nodes[i+1]):
                path_exists = False
                break

        # If path exists in the current graph, return True
        if path_exists:
            return True

    # If none of the graphs contain the path, return False
    return False

In [ ]:
from transformers import AutoConfig, GPT2Tokenizer
import os

# new config
TOKENIZER_PATH = "gv6_tokenizer"
CONFIG_SAVE_PATH = "shortest_v3_config_pdrop" 

print(f"Loading custom tokenizer from: {TOKENIZER_PATH}")
my_custom_tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER_PATH)


print("Loading base gpt2 config...")
config = AutoConfig.from_pretrained('gpt2')


print(f"Overriding config vocab_size. Old: {config.vocab_size}, New: {len(my_custom_tokenizer)}")
config.vocab_size = len(my_custom_tokenizer)
config.bos_token_id = my_custom_tokenizer.bos_token_id
config.eos_token_id = my_custom_tokenizer.eos_token_id
config.pad_token_id = my_custom_tokenizer.pad_token_id
print(f"Set pad_token_id to: {config.pad_token_id}")

new_dropout_rate = 0.5
print(f"Setting all dropout probabilities to: {new_dropout_rate}")
config.attn_pdrop = new_dropout_rate
config.resid_pdrop = new_dropout_rate
config.embd_pdrop = new_dropout_rate

os.makedirs(CONFIG_SAVE_PATH, exist_ok=True)
config.save_pretrained(CONFIG_SAVE_PATH)

print(f"\nCustom model config with high dropout saved successfully to: {CONFIG_SAVE_PATH}")

Loading custom tokenizer from: gv6_tokenizer
Loading base gpt2 config...
Overriding config vocab_size. Old: 50257, New: 1031
Set pad_token_id to: 0
Setting all dropout probabilities to: 0.5

Custom model config with high dropout saved successfully to: shortest_v3_config_pdrop


In [ ]:
import os

project_cache_path = os.path.join(os.getcwd(), '.cache', 'huggingface_datasets')
os.makedirs(project_cache_path, exist_ok=True)

os.environ['HF_DATASETS_CACHE'] = project_cache_path

print(f"Hugging Face datasets cache is now set to: {os.environ['HF_DATASETS_CACHE']}")

wandb_path = os.path.join(os.getcwd(), 'wandb_local_runs')
os.makedirs(wandb_path, exist_ok=True)

os.environ['WANDB_DIR'] = wandb_path
print(f"Wandb local run directory set to: {os.environ['WANDB_DIR']}")

import os
os.environ["WANDB__SERVICE_WAIT"] = "120"

Hugging Face datasets cache is now set to: /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/.cache/huggingface_datasets
Wandb local run directory set to: /cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/wandb_local_runs


In [ ]:
import os
import gc
import wandb

def train_model_script(num_epochs=1,
                       output_dir='sv3_model_ft',
                       lr=5e-05,
                       wandb_run_id=None):
    gc.collect()
    wandb.login()
    os.environ['WANDB_PROJECT'] = 'shortest_path_models'

    save_steps = 1000
    save_total_limit = 10
    weight_decay = 0.1
    beta2 = 0.98

    data_dir = 'sv3_data' 
    tokenizer_dir = 'gv6_tokenizer'
    config_dir = 'shortest_v3_config_pdrop'
    
    train_path = f'./{data_dir}/train.txt'
    test_path = f'./{data_dir}/test.txt'
    if wandb_run_id:
        print(f"Attempting to resume W&B run with ID: {wandb_run_id}")
        os.environ['WANDB_RESUME'] = "allow"
        os.environ['WANDB_RUN_ID'] = wandb_run_id

    command = f"""
    python3 ./train_shortest.py \\
        --config_name {config_dir} \\
        --tokenizer_name {tokenizer_dir} \\
        --train_file {train_path} \\
        --validation_file {test_path} \\
        --per_device_train_batch_size 128 \\
        --per_device_eval_batch_size 128\\
        --do_train \\
        --do_eval \\
        --output_dir {output_dir} \\
        --num_train_epochs {num_epochs} \\
        --save_strategy 'steps' \\
        --save_steps {save_steps} \\
        --save_total_limit {save_total_limit} \\
        --eval_strategy 'steps' \\
        --eval_steps 500 \\
        --warmup_steps 0 \\
        --learning_rate {lr} \\
        --report_to 'wandb' \\
        --lr_scheduler_type cosine \\
        --weight_decay {weight_decay} \\
        --adam_beta2 {beta2} \\
        --fp16 True
    """
    
    print("--- Running Command ---")
    print(command)
    print("-----------------------")
    
    os.system(command)

In [ ]:
#!rm -rf sv2_model_fixed/
!mkdir sv3_model_ft/

mkdir: cannot create directory ‘sv3_model_unm/’: File exists


In [ ]:


train_model_script(num_epochs=25,
                   output_dir='sv3_model_ft',
                   lr=1e-05, wandb_run_id="9p43e26h")


Attempting to resume W&B run with ID: 9p43e26h
--- Running Command ---

    python3 ./train_shortest.py \
        --config_name shortest_v3_config_pdrop \
        --tokenizer_name gv6_tokenizer \
        --train_file ./sv3_data/train.txt \
        --validation_file ./sv3_data/test.txt \
        --per_device_train_batch_size 128 \
        --per_device_eval_batch_size 128\
        --do_train \
        --do_eval \
        --output_dir sv3_model_ft \
        --num_train_epochs 25 \
        --save_strategy 'steps' \
        --save_steps 1000 \
        --save_total_limit 10 \
        --eval_strategy 'steps' \
        --eval_steps 500 \
        --warmup_steps 0 \
        --learning_rate 1e-05 \
        --report_to 'wandb' \
        --lr_scheduler_type cosine \
        --weight_decay 0.1 \
        --adam_beta2 0.98 \
        --fp16 True
    
-----------------------


/cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


INFO:__main__:Checkpoint detected, resuming training at sv3_model_ft/checkpoint-123500. To avoid this behavior, change the `--output_dir` or add `--overwrite_output_dir` to train from scratch.
INFO:__main__:Loading tokenizer, config, and model...
INFO:__main__:--- Initializing a new model from scratch ---
INFO:__main__:Synchronized model embedding size with tokenizer. New size: 1031


/cs/student/projects1/aibh/2024/cbaumgar/MSC_THESIS/./train_shortest.py:228: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


INFO:__main__:*** Starting Training ***


...

{'eval_loss': 0.10531479120254517, 'eval_token_accuracy': 0.9586219484371435, 'eval_exact_match_accuracy': 0.6338, 'eval_runtime': 12.3829, 'eval_samples_per_second': 807.568, 'eval_steps_per_second': 6.38, 'epoch': 18.81}


  0%|          | 0/79 [00:00<?, ?it/s]

{'loss': 0.0769, 'grad_norm': 1.5126792192459106, 'learning_rate': 1.4108449123128115e-05, 'epoch': 18.88}



 97%|█████████▋| 77/79 [00:11<00:00,  6.48it/s]
                                                           
100%|██████████| 79/79 [00:12<00:00,  6.55it/s]
                                               

{'eval_loss': 0.10346191376447678, 'eval_token_accuracy': 0.9585854437599818, 'eval_exact_match_accuracy': 0.6333, 'eval_runtime': 12.3671, 'eval_samples_per_second': 808.596, 'eval_steps_per_second': 6.388, 'epoch': 18.88}


In [ ]:
s = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i'}
s2 = {'c', 'd', 'a', 'b', 'i', 'g', 'f', 'h', 'e'}
list1 = list(s)
list2 = list(s2)
random.shuffle(list2)  # Shuffle to ensure different order
print(list1)  # This will print the elements in the set in a specific order
print(list2)  # This will print the elements in the set in a different order

print(list1 == list2) # This will be True within the same session

['h', 'a', 'd', 'e', 'f', 'i', 'g', 'b', 'c']
['g', 'e', 'f', 'b', 'd', 'c', 'i', 'h', 'a']
False


In [ ]:
# Define two sets with same elements but created from different orderings
random.seed(42)  # Ensure deterministic order
set1 = set(['a', 'b', 'c', 'd', 'e'])
set2 = set(['e', 'd', 'c', 'b', 'a'])

print("Set1:", set1)
print("Set2:", set2)

# Check if sets are equal
print("Sets equal?", set1 == set2)

# Convert sets to lists (order depends on internal hash)
list1 = list(set1)
list2 = list(set2)

print("List1:", list1)
print("List2:", list2)

# Are the lists equal (order and content)?
print("Lists equal (order and content)?", list1 == list2)


Set1: {'a', 'd', 'e', 'b', 'c'}
Set2: {'a', 'd', 'b', 'e', 'c'}
Sets equal? True
List1: ['a', 'd', 'e', 'b', 'c']
List2: ['a', 'd', 'b', 'e', 'c']
Lists equal (order and content)? False
